In [21]:
import csv
import pandas

添加额外的sequence信息到drug中

In [22]:
#读取drugcsv
drug_df = pandas.read_csv('data/drug.csv')
drug_df.head()

,id,name,type,groups,atc_codes,categories,smiles,pubmed_id,synonyms,inchi,inchikey,description,datasource
0,DB00001,Lepirudin,biotech,approved|withdrawn,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",NaN,16244762.0,"[Leu1, Thr2]-63-desulfohirudin|Desulfatohirudi...",NaN,NaN,Lepirudin is a recombinant hirudin formed by 6...,drugbank
1,DB00002,Cetuximab,biotech,approved,L01FE01,"Amino Acids, Peptides, and Proteins|Antibodies...",NaN,11752352.0,Cetuximab|Cétuximab|Cetuximabum,NaN,NaN,Cetuximab is a recombinant chimeric human/mous...,drugbank
2,DB00003,Dornase alfa,biotech,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",NaN,8792953.0,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,NaN,Dornase alfa is a biosynthetic form of human d...,drugbank
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",NaN,17187516.0,Denileukin|Denileukin diftitox|Interleukin-2/d...,NaN,NaN,A recombinant DNA-derived cytotoxic protein co...,drugbank
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,"Agents reducing cytokine levels|Amino Acids, P...",NaN,27463856.0,Etanercept|etanercept-szzs|etanercept-ykro|Rec...,NaN,NaN,Dimeric fusion protein consisting of the extra...,drugbank


In [23]:
#解析drug sequences.fasta
drug_seq = []
seq_dict = {}
seq = ''
with open('download/drug sequences.fasta') as f:
    for line in f:
        #直到找到下一个>，将上一个>的序列的下一行加入seq_dict
        if line.startswith('>'): 
            if seq: #如果seq不为空
                seq_dict[drug_seq[-1]] = seq #将上一个>的序列的下一行加入seq_dict
                seq = '' #清空seq
            drug_seq.append(line[1:].strip()) #将>后面的序列加入drug_seq
        else:
            seq += line.strip() #将序列的下一行加入seq
seq_dict[drug_seq[-1]] = seq #将最后一个>的序列的下一行加入seq_dict

In [24]:
# 将seq_dict转化为dataframe，分离信息为drug_id和sequence
seq_df = pandas.DataFrame.from_dict(seq_dict, orient='index')
seq_df = seq_df.reset_index()
seq_df.columns = ['drug_id', 'sequence']
seq_df

,drug_id,sequence
0,drugbank_drug|DB00001 DB00001 sequence,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...
1,drugbank_drug|DB00002 Cetuximab heavy chain,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...
2,drugbank_drug|DB00002 Cetuximab light chain,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...
3,drugbank_drug|DB00003 Dornase alfa sequence,LKIAAFNIQTFGETKMSNATLVSYIVQILSRYDIALVQEVRDSHLT...
4,drugbank_drug|DB00004 DB00004 sequence,MGADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGN...
...,...,...
433,drugbank_drug|DB16701 Human Plasminogen:,MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSI...
434,drugbank_drug|DB16755 Heavy Chain,QITLKESGPTLVKPTQTLTLTCTFSGFSLSISGVGVGWLRQPPGKA...
435,drugbank_drug|DB16755 Light Chain,QSALTQPASVSGSPGQSITISCTATSSDVGDYNYVSWYQQHPGKAP...
436,drugbank_drug|DB17012 sp|P04406|G3P_HUMAN OS=H...,MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYM...


In [25]:
drug_id = seq_df['drug_id']#提取seq_df中的drug_id
sequence = seq_df['sequence']#提取seq_df中的sequence
drug_id1 = drug_id.str.extract(r'(DB\d+)')#将drug_id中带有DB的字符提取出来
drug_seq_df = pandas.concat([drug_id1, sequence], axis=1)#合并drug_id1和sequence
drug_seq_df.columns = ['id', 'sequence']#重命名列名
drug_seq_df

,id,sequence
0,DB00001,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...
1,DB00002,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...
2,DB00002,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...
3,DB00003,LKIAAFNIQTFGETKMSNATLVSYIVQILSRYDIALVQEVRDSHLT...
4,DB00004,MGADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGN...
...,...,...
433,DB16701,MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSI...
434,DB16755,QITLKESGPTLVKPTQTLTLTCTFSGFSLSISGVGVGWLRQPPGKA...
435,DB16755,QSALTQPASVSGSPGQSITISCTATSSDVGDYNYVSWYQQHPGKAP...
436,DB17012,MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYM...


In [26]:
#将drug_seq_df信息加入drug_df
drug_df = pandas.merge(drug_df, drug_seq_df, on='id', how='left')
drug_df

,id,name,type,groups,atc_codes,categories,smiles,pubmed_id,synonyms,inchi,inchikey,description,datasource,sequence
0,DB00001,Lepirudin,biotech,approved|withdrawn,B01AE02,"Amino Acids, Peptides, and Proteins|Anticoagul...",NaN,16244762.0,"[Leu1, Thr2]-63-desulfohirudin|Desulfatohirudi...",NaN,NaN,Lepirudin is a recombinant hirudin formed by 6...,drugbank,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...
1,DB00002,Cetuximab,biotech,approved,L01FE01,"Amino Acids, Peptides, and Proteins|Antibodies...",NaN,11752352.0,Cetuximab|Cétuximab|Cetuximabum,NaN,NaN,Cetuximab is a recombinant chimeric human/mous...,drugbank,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...
2,DB00002,Cetuximab,biotech,approved,L01FE01,"Amino Acids, Peptides, and Proteins|Antibodies...",NaN,11752352.0,Cetuximab|Cétuximab|Cetuximabum,NaN,NaN,Cetuximab is a recombinant chimeric human/mous...,drugbank,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...
3,DB00003,Dornase alfa,biotech,approved,R05CB13,"Amino Acids, Peptides, and Proteins|Cough and ...",NaN,8792953.0,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,NaN,Dornase alfa is a biosynthetic form of human d...,drugbank,LKIAAFNIQTFGETKMSNATLVSYIVQILSRYDIALVQEVRDSHLT...
4,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,"ADP Ribose Transferases|Amino Acids, Peptides,...",NaN,17187516.0,Denileukin|Denileukin diftitox|Interleukin-2/d...,NaN,NaN,A recombinant DNA-derived cytotoxic protein co...,drugbank,MGADDVVDSSKSFVMENFSSYHGTKPGYVDSIQKGIQKPKSGTQGN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15401,DB17382,AUM-601,small molecule,investigational,NaN,NaN,NaN,NaN,"(R,E)-3-(5-(2-(2,5-difluorophenyl)pyrrolidin-1...",NaN,NaN,AUM-601 is a highly selective pan-TRK(tropomyo...,drugbank,NaN
15402,DB17383,FN-1501,small molecule,investigational,NaN,NaN,CN1CCN(CC2=CC=C(NC(=O)C3=NNC=C3NC3=C4C=CNC4=NC...,NaN,"4-((7h-pyrrolo (2,3-d)pyrimidin-4-yl)amino)-n-...",InChI=1S/C22H25N9O/c1-30-8-10-31(11-9-30)13-15...,VXLAKHWYGRKCGI-UHFFFAOYSA-N,NaN,drugbank,NaN
15403,DB17384,Tinengotinib,small molecule,investigational,NaN,NaN,CC1=C2N=C(C3=CC=CC=C3Cl)C3=C(NC2=NN1)C=C(N=C3)...,NaN,"4-(5-(2-Chlorophenyl)-3-Methyl-2,10-Dihydropyr...",InChI=1S/C20H19ClN6O/c1-12-18-20(26-25-12)23-1...,DQFCVOOFMXEPOC-UHFFFAOYSA-N,NaN,drugbank,NaN
15404,DB17385,Lipotecan,small molecule,investigational,NaN,Alkaloids,CC[C@@]1(OC(=O)C(C)ON=C2C3=C(C4=C2C=C(C=C4[N+]...,NaN,"Lipothecan free base|Propanoic acid, 2-(((2,4,...",InChI=1S/C39H30N8O15/c1-5-39(26-13-30-34-18(14...,JCCCLGDYMMTBPM-HXDHBHDHSA-N,NaN,drugbank,NaN


In [27]:
#输出drug_df为csv
drug_df.to_csv('drug_ex.csv', index=False)

将target的senquence分类为gene和protein，加入target_df中

In [28]:
#读取fasta文件并存为dataframe
def read_fasta(file):
    seq_dict = {}
    seq = ''
    with open(file) as f:
        for line in f:
            if line.startswith('>'): 
                if seq: 
                    seq_dict[drug_seq[-1]] = seq 
                    seq = '' 
                drug_seq.append(line[1:].strip()) 
            else:
                seq += line.strip() 
        seq_dict[drug_seq[-1]] = seq 
    seq_df = pandas.DataFrame.from_dict(seq_dict, orient='index')
    seq_df = seq_df.reset_index()
    seq_df.columns = ['drug_id', 'sequence']
    drug_id = seq_df['drug_id']
    sequence = seq_df['sequence']
    drug_id1 = drug_id.str.extract(r'(DB\d+)')
    seq_df = pandas.concat([drug_id1, sequence], axis=1)
    seq_df.columns = ['drugbank_id', 'sequence']

    return seq_df

In [29]:
file1='download/gene/carrier_gene.fasta'
file2='download/gene/enzyme_gene.fasta'
file3='download/gene/target_gene.fasta'
file4='download/gene/transporter_gene.fasta'
seq_df1 = read_fasta(file1)
seq_df2 = read_fasta(file2)
seq_df3 = read_fasta(file3)
seq_df4 = read_fasta(file4)

In [30]:
#合并多个dataframe
tar_seq=pandas.concat([seq_df1,seq_df2,seq_df3,seq_df4],ignore_index=True)
#新增列名
tar_seq['tartype']='gene'
tar_seq

,drugbank_id,sequence,tartype
0,DB09130,ATGCAAATAGAGCTCTCCACCTGCTTCTTTCTGTGCCTTTTGCGAT...,gene
1,DB09130,ATGTTCCCAGGCTGCCCACGCCTCTGGGTCCTGGTGGTCTTGGGCA...,gene
2,DB06784,ATGAGCTCCCAGATTCGTCAGAATTATTCCACCGACGTGGAGGCAG...,gene
3,DB09140,ATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGG...,gene
4,DB00180,ATGCCACTCCTCCTGTACACCTGTCTTCTCTGGCTGCCCACCAGCG...,gene
...,...,...,...
5976,DB09462,ATGGTTCAAGCATCCGGGCACAGGCGGTCCACCCGTGGCTCCAAAA...,gene
5977,DB09462,ATGCAGCCTGAGGGAGCAGAAAAGGGAAAAAGCTTCAAGCAGAGAC...,gene
5978,DB00107,ATGGCTGCCGGAACAGCAGTTGGAGCCTGGGTGCTGGTCCTCAGTC...,gene
5979,DB00640,ATGGCCGTTGTCTCAGAGGACGACTTTCAGCACAGTTCAAACTCCA...,gene


In [31]:
file5='download/protein/carrier_protein.fasta'
file6='download/protein/enzyme_protein.fasta'
file7='download/protein/target_protein.fasta'
file8='download/protein/transporter_protein.fasta'
seq_df5 = read_fasta(file5)
seq_df6 = read_fasta(file6)
seq_df7 = read_fasta(file7)
seq_df8 = read_fasta(file8)

In [32]:
#合并多个dataframe
tar_seq1=pandas.concat([seq_df5,seq_df6,seq_df7,seq_df8],ignore_index=True)
#新增列名
tar_seq1['tartype']='protein'
tar_seq1

,drugbank_id,sequence,tartype
0,DB09130,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,protein
1,DB09130,MFPGCPRLWVLVVLGTSWVGWGSQGTEAAQLRQFYVAAQGISWSYR...,protein
2,DB06784,MSSQIRQNYSTDVEAAVNSLVNLYLQASYTYLSLGFYFDRDDVALE...,protein
3,DB09140,MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPH...,protein
4,DB00180,MPLLLYTCLLWLPTSGLWTVQAMDPNAAYVNMSNHHRGLASANVDF...,protein
...,...,...,...
6278,DB09462,MVQASGHRRSTRGSKMVSWSVIAKIQEILQRKMVREFLAEFMSTYV...,protein
6279,DB09462,MQPEGAEKGKSFKQRLVLKSSLAKETLSEFLGTFILIVLGCGCVAQ...,protein
6280,DB00107,MAAGTAVGAWVLVLSLWGAVVGAQNITARIGEPLVLKCKGAPKKPP...,protein
6281,DB00640,MAVVSEDDFQHSSNSTYRTTSSSLRADQEALLEKLLDRPPPGLQRP...,protein


In [33]:
tar_seq2=pandas.concat([tar_seq,tar_seq1],ignore_index=True)

#输出为csv
tar_seq2.to_csv('data/tar_seq.csv', index=False)
tar_seq2

,drugbank_id,sequence,tartype
0,DB09130,ATGCAAATAGAGCTCTCCACCTGCTTCTTTCTGTGCCTTTTGCGAT...,gene
1,DB09130,ATGTTCCCAGGCTGCCCACGCCTCTGGGTCCTGGTGGTCTTGGGCA...,gene
2,DB06784,ATGAGCTCCCAGATTCGTCAGAATTATTCCACCGACGTGGAGGCAG...,gene
3,DB09140,ATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGG...,gene
4,DB00180,ATGCCACTCCTCCTGTACACCTGTCTTCTCTGGCTGCCCACCAGCG...,gene
...,...,...,...
12259,DB09462,MVQASGHRRSTRGSKMVSWSVIAKIQEILQRKMVREFLAEFMSTYV...,protein
12260,DB09462,MQPEGAEKGKSFKQRLVLKSSLAKETLSEFLGTFILIVLGCGCVAQ...,protein
12261,DB00107,MAAGTAVGAWVLVLSLWGAVVGAQNITARIGEPLVLKCKGAPKKPP...,protein
12262,DB00640,MAVVSEDDFQHSSNSTYRTTSSSLRADQEALLEKLLDRPPPGLQRP...,protein


In [34]:
#读取target.csv
target_df = pandas.read_csv('data/targetdf.csv')
#更换列顺序
target_df = target_df[['drugbank_id', 'target_name', 'id', 'category', 'organism', 'known_action','actions','pubmed_id', 'uniprot_id']]
target_df

,drugbank_id,target_name,id,category,organism,known_action,actions,pubmed_id,uniprot_id
0,DB00001,Epidermal growth factor receptor,BE0000767,target,Humans,yes,inhibitor,10505536.0,P00734
1,DB00002,Low affinity immunoglobulin gamma Fc region re...,BE0000901,target,Humans,yes,binder,11752352.0,P00533
2,DB00002,Complement C1q subcomponent subunit A,BE0002094,target,Humans,unknown,binder,16336752.0,O75015
3,DB00002,Complement C1q subcomponent subunit B,BE0002095,target,Humans,unknown,binder,32117299.0,P02745
4,DB00002,Complement C1q subcomponent subunit C,BE0002096,target,Humans,unknown,binder,32117299.0,P02746
...,...,...,...,...,...,...,...,...,...
28320,DB17083,Solute carrier family 22 member 6,BE0001066,transporter,Humans,no,substrate,NaN,Q9UNQ0
28321,DB17083,Histamine H3 receptor,BE0000968,transporter,Humans,no,inhibitor,NaN,Q4U2R8
28322,DB17087,Plasmepsin X,BE0010203,target,Humans,yes,antagonist,28424564.0,Q9Y5N1
28323,DB17096,Plasmepsin X,BE0010204,target,Plasmodium falciparum (isolate 3D7),yes,inhibitor|binder,36216349.0,Q8IAS0


In [35]:
#去重
tar_seq3 = tar_seq2.drop_duplicates(subset=['drugbank_id'], keep='first')
tar_seq3

,drugbank_id,sequence,tartype
0,DB09130,ATGCAAATAGAGCTCTCCACCTGCTTCTTTCTGTGCCTTTTGCGAT...,gene
2,DB06784,ATGAGCTCCCAGATTCGTCAGAATTATTCCACCGACGTGGAGGCAG...,gene
3,DB09140,ATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGG...,gene
4,DB00180,ATGCCACTCCTCCTGTACACCTGTCTTCTCTGGCTGCCCACCAGCG...,gene
5,DB00162,ATGATGAGAGAATGGGTTCTGCTCATGTCCGTGCTGCTCTGTGGCC...,gene
...,...,...,...
11967,DB16258,MELLILKANAITTILTAVTFCFASGQNITEEFYQSTCSAVSKGYLS...,protein
12115,DB00103,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...,protein
12177,DB09363,PIIPSYLYSIKHEKNATEIQTARPVHTASISDSFQSIFSYDNSTMV...,protein
12206,DB13178,MRAVLDTADIAIVALYFILVMCIGFFAMWKSNRSTVSGYFLAGRSM...,protein


In [36]:
#将target_df中的drugbank_id和tar_seq2中的drugbank_id合并
target_df = pandas.merge(target_df, tar_seq3, on='drugbank_id', how='left')
target_df

,drugbank_id,target_name,id,category,organism,known_action,actions,pubmed_id,uniprot_id,sequence,tartype
0,DB00001,Epidermal growth factor receptor,BE0000767,target,Humans,yes,inhibitor,10505536.0,P00734,ATGGCGCACGTCCGAGGCTTGCAGCTGCCTGGCTGCCTGGCCCTGG...,gene
1,DB00002,Low affinity immunoglobulin gamma Fc region re...,BE0000901,target,Humans,yes,binder,11752352.0,P00533,ATGTGGTTCTTGACAACTCTGCTCCTTTGGGTTCCAGTTGATGGGC...,gene
2,DB00002,Complement C1q subcomponent subunit A,BE0002094,target,Humans,unknown,binder,16336752.0,O75015,ATGTGGTTCTTGACAACTCTGCTCCTTTGGGTTCCAGTTGATGGGC...,gene
3,DB00002,Complement C1q subcomponent subunit B,BE0002095,target,Humans,unknown,binder,32117299.0,P02745,ATGTGGTTCTTGACAACTCTGCTCCTTTGGGTTCCAGTTGATGGGC...,gene
4,DB00002,Complement C1q subcomponent subunit C,BE0002096,target,Humans,unknown,binder,32117299.0,P02746,ATGTGGTTCTTGACAACTCTGCTCCTTTGGGTTCCAGTTGATGGGC...,gene
...,...,...,...,...,...,...,...,...,...,...,...
28320,DB17083,Solute carrier family 22 member 6,BE0001066,transporter,Humans,no,substrate,NaN,Q9UNQ0,NaN,NaN
28321,DB17083,Histamine H3 receptor,BE0000968,transporter,Humans,no,inhibitor,NaN,Q4U2R8,NaN,NaN
28322,DB17087,Plasmepsin X,BE0010203,target,Humans,yes,antagonist,28424564.0,Q9Y5N1,NaN,NaN
28323,DB17096,Plasmepsin X,BE0010204,target,Plasmodium falciparum (isolate 3D7),yes,inhibitor|binder,36216349.0,Q8IAS0,ATGAAACGCATTAGCCCTCTAAACACCCTTTTTTATTTAAGTTTAT...,gene


In [37]:
target_df=target_df[['id', 'target_name', 'drugbank_id', 'category', 'organism', 'known_action','actions','pubmed_id', 'uniprot_id','tartype','sequence']]#调换列的顺序
target_df = target_df.drop(['drugbank_id'], axis=1)#删除drugbank_id
target_df['datasource']='drugbank'#新增datasource列

#输出target_df为csv
target_df.to_csv('target_ex.csv', index=False)